In [1]:
import os


In [2]:
%pwd

'c:\\Users\\subik\\OneDrive\\Document\\PROJECT\\End-to-end-ML-project-using-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\subik\\OneDrive\\Document\\PROJECT\\End-to-end-ML-project-using-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen =True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
from src.mlproject.constants import *


In [11]:
from src.mlproject.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
                 self.config = read_yaml(config_filepath)
                 self.params = read_yaml(params_filepath)
                 self.schema = read_yaml(schema_filepath)
                 
                 create_directories([self.config["artifacts_root"]])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]
        create_directories([config["root_dir"]])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config["root_dir"]),
            source_URL = config["source_URL"],
            local_data_file = Path(config["local_data_file"]),
            unzip_dir = Path(config["unzip_dir"])
        )
        
        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
      
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
      

In [27]:
import yaml
from box import Box

with open("config.yaml", "r") as f:
    data = yaml.safe_load(f)
    print("Raw YAML data:", data)
    config = Box(data or {})
    print("Box config:", config)


FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-20 22:54:38,399]: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-20 22:54:38,407]: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-20 22:54:38,412]: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-20 22:54:38,414]: INFO: common: created directory at: artifacts]
[2025-09-20 22:54:38,419]: INFO: common: created directory at: artifacts/data_ingestion]


[2025-09-20 22:54:40,237]: INFO: 3819233211: artifacts\data_ingestion\data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4AE8:38B866:126B98:2F133A:68CEE356
Accept-Ranges: bytes
Date: Sat, 20 Sep 2025 17:24:39 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10225-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1758389079.875479,VS0,VE342
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: f900e721faaf516caa88316f7113b649b97a089a
Expires: Sat, 20 Sep 2025 17:29:39 GMT
Source-Age: 0

]
